In [1]:
# installing required libraries 
from IPython.display import clear_output

!pip install fiftyone
!pip install import-ipynb
clear_output()

In [2]:
# importing necessary libraries 
import fiftyone 
import pandas as pd
import json 
import matplotlib.pyplot as plt
import cv2
import os 
from PIL import Image
import shutil
import import_ipynb

Migrating database to v0.17.2


INFO:fiftyone.migrations.runner:Migrating database to v0.17.2


In [3]:
# mount google colab to drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/MyDrive/object_detection
import utils
%cd /content

/content/gdrive/MyDrive/object_detection
importing Jupyter notebook from utils.ipynb
/content


In [5]:
# importing 150 images from coco-2017 dataset using fiftyone library. Selected only class 'person' for detection. 
datasets = fiftyone.zoo.load_zoo_dataset("coco-2017", split = 'train',
label_types = ['detections'], classes = ['person'], max_samples = 150)
clear_output()

In [9]:
# reading labels.json file --> ['info', 'licences', 'images', 'annotations', 'categories']
with open('/root/fiftyone/coco-2017/train/labels.json', 'r') as file:
  data = json.load(file)

In [10]:
# create a dataframe containing information about the images including bounding boxes, image_id, image_path
image_id = []
bbox = []
for i in range(len(data['annotations'])):
  if data['annotations'][i]['category_id'] == 1: # here 1 refers to category person
    image_id.append(data['annotations'][i]['image_id']) # to get image_id
    bbox.append(data['annotations'][i]['bbox']) # to get bounding box 

dataframe = pd.DataFrame() 
dataframe['image_id'] = image_id
dataframe['bbox'] = bbox

dataframe.sort_values(by = 'image_id', ascending = True, inplace = True)
dataframe.reset_index(drop = True, inplace = True)

image_path = dict()
for i in data['images']: # to get image path
  image_path[i['id']] = '/root/fiftyone/coco-2017/validation/data/' + i['file_name']

dataframe['image_path'] = dataframe['image_id'].map(image_path)

In [11]:
# Dividing the dataset for training, validation, and testing. 
train_size = int(0.7 * len(datasets)) 
valid_size = int(0.2 * len(datasets))

train_num = dataframe['image_id'].value_counts().sort_index(ascending = True)[:train_size].sum()
valid_num = dataframe['image_id'].value_counts().sort_index(ascending = True)[train_size: train_size + valid_size].sum()

training_data = dataframe.loc[:train_num] # selecting 70% of images for training
validation_data = dataframe.loc[train_num + 1: train_num + valid_num] # selecting 20% of images for validation
testing_data = dataframe.loc[train_num + valid_num + 1: ] # selecting 10% of images for testing

In [ ]:
def create_images(dataframe):
  '''input: dataframe with 
    output: regions with labels as 0 for background and 1 for object'''
  images = []
  labels = []
  for i in range(len(dataframe)):
    try:
      image_path = dataframe.loc[i, 'image_path']
      bb = dataframe.iloc[i, 1]

      cv2.setUseOptimized(True);
      ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation() # instantiate selective search segmentation algorithm 

      im = cv2.imread(image_path) 
      if len(im.shape) == 2: # if an image is a grayscale(channel=1), convert it to 3 channels. 
        im = cv2.merge(im, im, im)
      ss.setBaseImage(im) 
      ss.switchToSelectiveSearchFast()
      rects = ss.process() # rects contains bounding box(top_left_x_coordinate, top_left_y_coordinate, width, height) of all the regions. 

      positive_count = 0
      negative_count = 0
      for rect in (rects):
        imOut = im.copy()
        x, y, w, h = rect # bounding box of each region
        iou = utils.calculate_iou(bb, rect) # calculate IOU of each region with the ground truth bounding box/original object's bounding box
        if (positive_count < 20) and (iou > 0.7): # to get 20 object regions from the image with IOU > 0.7
            imOut = imOut[y:y+h, x:x+w, :] # cropped object from the image 
            resized_image = cv2.resize(imOut, (224, 224), interpolation = cv2.INTER_AREA)
            images.append(resized_image)
            labels.append(1)
            positive_count += 1
        
        if (negative_count <20) and (iou < 0.3): # to get 20 background regions from the image with IOU < 0.3
          imOut = imOut[y:y+h, x:x+w, :] # cropped background from the image
          resized_image = cv2.resize(imOut, (224, 224), interpolation = cv2.INTER_AREA)
          images.append(resized_image)
          labels.append(0)
          negative_count += 1      

    except Exception as e:
      print(image_path, '\n')
      continue

  return images, labels



In [ ]:
training_images, training_labels = create_images(training_data) # creating training objects and background 

# create a folder for training images with folders containing objects and background

pos_path = 'gdrive/MyDrive/object_detection/training_images'
if os.path.exists(pos_path): # if path already exists, delete the directory and create new directory
  shutil.rmtree(pos_path) 
os.mkdir(pos_path)

training_pos_path = 'gdrive/MyDrive/object_detection/training_images/object'
os.mkdir(training_pos_path)

training_neg_path = 'gdrive/MyDrive/object_detection/training_images/noise'
os.mkdir(training_neg_path)

utils.save_images(training_images, training_labels, training_pos_path, training_neg_path) # save images 

In [ ]:
validation_data.reset_index(drop = True, inplace = True)

validation_images, validation_labels = create_images(validation_data) # creating validation objects and background 

# create a folder for validation images with folders containing objects and background 

neg_path = 'gdrive/MyDrive/object_detection/validation_images' 
if os.path.exists(neg_path):
  shutil.rmtree(neg_path)
os.mkdir(neg_path)

validation_pos_path = 'gdrive/MyDrive/object_detection/validation_images/object'
os.mkdir(validation_pos_path)

validation_neg_path = 'gdrive/MyDrive/object_detection/validation_images/noise'
os.mkdir(validation_neg_path)

utils.save_images(validation_images, validation_labels, validation_pos_path, validation_neg_path)

In [ ]:
# saving test images 
test_images_path = testing_data['image_path'].unique()

test_path = 'gdrive/MyDrive/object_detection/test_images'

if os.path.exists(test_path):
  shutil.rmtree(test_path) 
os.makedirs(test_path)

for index, image in enumerate(test_images_path):

  image = Image.fromarray(plt.imread(image))
  image.save(f'{test_path}/{index}.png')